In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
df_train = pd.read_csv('./fashionmnist/fashion-mnist_train.csv')
df_test = pd.read_csv('./fashionmnist/fashion-mnist_test.csv')

In [4]:
df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
len(df_train)

60000

In [6]:
y_train = df_train['label'].to_numpy()
x_train = df_train.drop('label', axis=1).to_numpy().reshape(-1, 28, 28, 1) / 255.0

In [7]:
y_test = df_test['label'].to_numpy()
x_test = df_test.drop('label', axis=1).to_numpy().reshape(-1, 28, 28, 1) / 255.0

In [8]:
def to_one_hot(label, depth):
    n = len(label)
    output = np.zeros((n, depth))
    output[range(n), label] = 1
    return output

In [9]:
y_train = to_one_hot(y_train, 10)
y_test = to_one_hot(y_test, 10)

In [10]:
X = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [11]:
drop_rate = tf.placeholder(tf.float32, shape=None)

In [12]:
def init_weights(shape):
    rand_norm_dist = tf.truncated_normal(shape, stddev=0.1)
    W = tf.Variable(rand_norm_dist)
    return W

def init_biases(shape):
    init_val = tf.constant(0.1, shape=shape)
    b = tf.Variable(init_val)
    return b

In [13]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pooling_2by2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [14]:
def convolutional_layer(x, filter_shape):
    W = init_weights(filter_shape)
    b = init_biases([filter_shape[3]])
    conv = tf.nn.relu(conv2d(x, W))
    return max_pooling_2by2(conv)

In [15]:
def normal_full_layer(x, output_size):
    input_size = int(x.get_shape()[1])
    W = init_weights([input_size, output_size])
    b = init_biases([output_size])
    return tf.matmul(x, W) + b

In [16]:
def drop_out(x, drop_rate):
    return tf.nn.dropout(x, rate=drop_rate)

In [17]:
conv1 = convolutional_layer(X, [5,5,1,32])
conv2 = convolutional_layer(conv1, [5,5,32,64])
conv2_flattened = tf.reshape(conv2, shape=[-1, 7*7*64])

norm1_pre_act = normal_full_layer(conv2_flattened, 1024)
norm1 = tf.nn.relu(norm1_pre_act)
norm1_dropped = drop_out(norm1, drop_rate)

y_pred = normal_full_layer(norm1_dropped, 10)

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y_pred)

In [19]:
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cross_entropy)

In [20]:
matches = tf.equal(tf.argmax(y_true, axis=1), tf.argmax(y_pred, axis=1))
accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))

In [21]:
init = tf.global_variables_initializer()

In [22]:
steps = 1000
batch_size = 20

with tf.Session() as sess:
    sess.run(init)
    
    i = 0
    for step in range(steps):
        x_batch, y_batch = x_train[i:i+batch_size], y_train[i:i+batch_size]
        i += batch_size
        
        sess.run(train, feed_dict={X: x_batch, y_true: y_batch, drop_rate: 0.5})
        
        if step%100 == 0:
            acc = sess.run(accuracy, feed_dict={X: x_test, y_true: y_test, drop_rate: 0.0})
            
            print("ON STEP {}\n ACCURACY: {}".format(step, acc))

ON STEP 0
 ACCURACY: 0.12800000607967377
ON STEP 100
 ACCURACY: 0.7861999869346619
ON STEP 200
 ACCURACY: 0.8101000189781189
ON STEP 300
 ACCURACY: 0.8015999794006348
ON STEP 400
 ACCURACY: 0.8220999836921692
ON STEP 500
 ACCURACY: 0.8097000122070312
ON STEP 600
 ACCURACY: 0.8629000186920166
ON STEP 700
 ACCURACY: 0.8597999811172485
ON STEP 800
 ACCURACY: 0.8413000106811523
ON STEP 900
 ACCURACY: 0.8610000014305115
